In [1]:
%reload_ext autoreload
%autoreload 2

import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()

import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 
from accelerate import Accelerator
import torch.nn.functional as F
# from einops import rearrange, pack, unpack
import sys
import os
#get cur dir
cur_dir = os.getcwd()
if 'miu' in cur_dir:
    sys.path.append('/home/miu/Projects/NLPLab/')
elif 'kiki' in cur_dir:
    sys.path.append('/home/kiki/dados_servidor/Servidor/NLPLab/')
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"


2.1.2


/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_training='hf'#'bert' , 'tokemonster
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#com retnet. Confirmado que é O(n). Se lembrando que não existe mta vantagem além do custo crescer sequencialmente, já que o conteudo 
#(4090) Para 84 tokens(tokenmonster salva 35%) batch size 200 e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias
#(4090) Para 168 tokens(tokenmonster salva 35%) batch size 100 e 16000 vocab (Electra): 1000 = 3.7s, 10000 = 37.5s,
#(4090) Para 1000 tokens(tokenmonster salva 35%) batch size 12 e 16000 vocab (Electra):            , 10000 = 4min32s,


#HIPOTÉTICO(Se conseguisse representar um texto com menos tokens):
#(4090) Para 24 tokens(tokenmonster salva 35%)(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias
#HIPOTÉTICO(Se diminuisse o vocabulario para 1024 tokens diferentes)
#(4090) Para 84 tokens(tokenmonster salva 35%) e 1024 vocab: 1000 =1.6s , 10000 = 16.2s,

#losses:
#1m dataset, electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#10m dataset, 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra) iteration:  49980 , total_loss:  20.8733185450236. 339min total


LIMIT_DATASET = 10_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
# VOCAB_SIZE = 32_768  # from Cramming
# DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
# MODEL_MAX_SEQ_LEN = 128  # from Cramming

VOCAB_SIZE = 32_768  # tokenmonster
# VOCAB_SIZE = 1_024  # tokenmonster
DEVICE_BATCH_SIZE = 12 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
MODEL_MAX_SEQ_LEN = 512 # token_monster


MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')

from utils.process_tokenizer import get_tokenizer
# tokenizer,tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'clm',MODEL_MAX_SEQ_LEN+2)
tokenized_dataset,norm,vocab,tokenizer=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'mlm',MODEL_MAX_SEQ_LEN+2)
# tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'smlm',MODEL_MAX_SEQ_LEN+2)
MASK_ID=vocab.tokenize("[MASK]")[0]
N_DOMAINS=3


batch_size:  2040


Resolving data files: 100%|██████████| 53/53 [00:00<00:00, 91.88it/s]


loaded dataset!!


## PreTrain

In [7]:
from x_transformers import TransformerWrapper, Decoder,Encoder


#encoder decoder
colt5_dict={
            'encoder':{'is_colt5_encoder':True,'light_ff_mult':0.5,'heavy_ff_mult':4,'num_heavy_tokens':30,
                   'light_dim_head':64,'light_heads':8,'light_window_size':128,
                   'heavy_dim_head':64,'heavy_heads':8,'num_heavy_tokens_q':30,
                   'num_heavy_tokens_kv':30},
            }

# model = TransformerWrapper(
model = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN,
    attn_layers = Encoder(
        dim = int(768),
        depth = 12,
        heads =12,
        attn_flash = True,
        colt5_dict=colt5_dict['encoder'],
        # rel_pos_bias = True
    ),
    emb_dim=128,
)

In [9]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence

train_dataloader = DataLoader(
        tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
    )
from utils.utils import get_optimizer_scheduler
optimizer,lr_scheduler,max_train_steps = get_optimizer_scheduler(model,train_dataloader,gradient_accumulation_steps,learning_rate=5e-5,weight_decay=0, num_warmup_steps=0, max_train_steps=None,lr_scheduler_type='linear',num_train_epochs=1)

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

In [10]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):

        # with accelerator.accumulate(model):
       
        logits=model(batch['input_ids']) 
        loss = F.cross_entropy(logits.transpose(1, 2),batch['input_ids'],ignore_index = PAD_ID)
        count_amostra+=int(len(batch['input_ids']))
        # print(loss)
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()       
        
        if step%steps_log==0:
            print('iteration: ',step,', total_loss: ',total_loss/steps_log)
            print('count_amostra:',count_amostra)
            total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1

    if completed_steps >= max_train_steps:
        break

    model.eval()
    losses = []


iteration:  0 , total_loss:  0.3496361414591471
count_amostra: 12


KeyboardInterrupt: 

In [ ]:
53s

In [6]:

#convert to huggingface Transformers

from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast

config = BertConfig(
    vocab_size=VOCAB_SIZE+8,
    max_position_embeddings=MODEL_MAX_SEQ_LEN,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=2,
)

bert_model=BertForMaskedLM(config=config)
bert_model.bert=model

In [7]:
bert_model.save_pretrained('models/bert')

## Evaluate

### QA

### Trainer hf

In [3]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [4]:
max_length = MODEL_MAX_SEQ_LEN#384
stride = 128
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [5]:
validation_dataset = dataset["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["test"].column_names,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4972.96 examples/s]


In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=tokenizer.model_max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        # print('sequence_ids: ',sequence_ids)
        # print('sequence_ids: ',len(sequence_ids))

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1 and idx < len(sequence_ids) - 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
from transformers import DefaultDataCollator

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names,batch_size=4)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 1404.71 examples/s]


In [13]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

#BertForMaskedLM
from transformers import BertForMaskedLM

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)
# bert_model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path='bert-base-uncased')
# bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
# bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert_spanbert")
if 'miu' in cur_dir:
    bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/miu/Projects/NLPLab/berts/models/bert")
else:
    bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /home/miu/Projects/NLPLab/berts/models/bert and are newly initialized: ['bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.attention.output.dense.bias', 'bert.encoder.layer.1.attention.output.dense.weight', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.8.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.8.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.output.dense.weight', 'bert.encoder.layer.2.attention.self.query.weight', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.4.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.value.bias', 'bert.encoder.layer.5.attention.self.query.weight', 'qa_outputs.bias', 'bert.encoder.layer.8.attention.sel

In [14]:
trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abcp4. Use `wandb login --relogin` to force relogin


 20%|██        | 500/2500 [00:39<02:38, 12.61it/s]

{'loss': 4.7581, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


 40%|████      | 1000/2500 [01:20<01:57, 12.82it/s]

{'loss': 4.0755, 'learning_rate': 1.2e-05, 'epoch': 2.0}


...

100%|██████████| 2500/2500 [03:21<00:00, 12.80it/s]

{'loss': 3.3212, 'learning_rate': 8e-09, 'epoch': 5.0}


100%|██████████| 2500/2500 [03:22<00:00, 12.37it/s]

{'train_runtime': 206.7869, 'train_samples_per_second': 96.718, 'train_steps_per_second': 12.09, 'train_loss': 3.8819138671875, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=3.8819138671875, metrics={'train_runtime': 206.7869, 'train_samples_per_second': 96.718, 'train_steps_per_second': 12.09, 'train_loss': 3.8819138671875, 'epoch': 5.0})

In [15]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate

metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [16]:
import evaluate

predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["test"])

100%|██████████| 1000/1000 [00:00<00:00, 1607.05it/s]


{'exact_match': 3.9, 'f1': 8.451318538500729}

100%|██████████| 1000/1000 [00:00<00:00, 1573.49it/s]


{'exact_match': 4.2, 'f1': 8.794485888565596}

In [ ]:
#100k 512 tokens
{'exact_match': 4.2, 'f1': 8.794485888565596}

## SEQ

In [6]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [7]:
model_extractor = HiddenLayerExtractor(model,-2)

In [8]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # self.l1 = model._modules['model']
        self.l1 = model_extractor
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.l3 = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 3)
    
    def forward(self, ids):
        output= self.l1(ids)
        # print('l1:',output.shape)
        output = self.l2(output)
        # print('l2:',output.shape)
        output = self.fl(output)
        # print('fl:',output.shape)
        output = self.l3(output)
        # print('l3:',output.shape)
        return output

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')
optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=2e-5,weight_decay=0.01)

In [9]:
task='mrpc'
dataset = load_dataset('glue', task, split='train')

In [10]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.ls=[]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        s=norm.normalize_str(self.dataset["sentence1"][i])+'[SEP]'+norm.normalize_str(self.dataset["sentence2"][i])
        tokens = vocab.tokenize(s).tolist()
        
        self.ls.append(i)
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)
        tokens=torch.Tensor(tokens).long()

        label=self.dataset["label"][i]
        
        d={'input_ids':tokens,'label':label}
        return d



from torch.utils.data import DataLoader

tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [11]:
import numpy as np
from torch.autograd import Variable

classifier_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs, targets)
    return loss

In [12]:
#FINETUNING

epochs=5
running_loss=0
for i in range(epochs):
    for _,data in enumerate(train_dataloader):
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        targets = data['label'].to('cuda', dtype = torch.long)

        outputs = classifier_model(ids)
        
        loss = loss_fn(outputs,targets)
        # loss=outputs.loss

        optimizer.zero_grad()
        running_loss+=loss.item()
        if _%50==0:
            print(f'Epoch: {i}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0, Loss:  0.030146934986114503
Epoch: 1, Loss:  0.5000924682617187
Epoch: 2, Loss:  0.5059760808944702
Epoch: 3, Loss:  0.4948312652111053
Epoch: 4, Loss:  0.48995151281356814


In [13]:
#EVALUATE
dataset = load_dataset('glue', task, split='validation')
tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [14]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)
    
    outputs = classifier_model(ids)
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

In [15]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.60      0.07      0.12       129
           1       0.69      0.98      0.81       279

    accuracy                           0.69       408
   macro avg       0.65      0.52      0.47       408
weighted avg       0.66      0.69      0.60       408



In [ ]:
#Bert 3 layers  10000 dataset
#               precision    recall  f1-score   support

#            0       0.61      0.09      0.15       129
#            1       0.70      0.97      0.81       279

#     accuracy                           0.69       408
#    macro avg       0.65      0.53      0.48       408
# weighted avg       0.67      0.69      0.60       408

## DensePhrases

In [3]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [4]:
#load model and tokenizer

from transformers import AutoTokenizer, AutoModel,AutoModelForQuestionAnswering
from transformers import TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

In [5]:
from transformers import PreTrainedModel
from transformers import AutoConfig
import copy
from torch.nn import CrossEntropyLoss
# Encoders: three LMs
config = AutoConfig.from_pretrained("bert-base-uncased")

class Encoder(PreTrainedModel):
    def __init__(self,return_phrase=False,
                 return_query=False):
        super().__init__(config)
        self.phrase_encoder = AutoModel.from_pretrained("bert-base-uncased")
        self.query_start_encoder = copy.deepcopy(self.phrase_encoder)
        self.query_end_encoder = copy.deepcopy(self.phrase_encoder)
        # Additional parameters
        self.filter_linear = nn.Linear(config.hidden_size, 2)
        self.return_phrase = return_phrase
        self.return_query = return_query

    def embed_phrase(self, input_ids, attention_mask, token_type_ids):
        
        """ Get phrase embeddings (token-wise) """
        outputs = self.phrase_encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return outputs[0], outputs[0]

    def embed_query(self, input_ids_, attention_mask_, token_type_ids_):
        """ Get query start/end embeddings """
        # print('input_ids_.shape: ',input_ids_.shape)
        # Two LM based query reps
        outputs_s_ = self.query_start_encoder(
            input_ids_,
            attention_mask=attention_mask_,
            token_type_ids=token_type_ids_,
        )
        outputs_e_ = self.query_end_encoder(
            input_ids_,
            attention_mask=attention_mask_,
            token_type_ids=token_type_ids_,
        )
        sequence_output_s_ = outputs_s_[0]
        sequence_output_e_ = outputs_e_[0]
        query_start = sequence_output_s_[:,:1,:]
        query_end = sequence_output_e_[:,:1,:]
        return query_start, query_end
    
    def forward(self,
            input_ids=None, attention_mask=None, token_type_ids=None,
            input_ids_=None, attention_mask_=None, token_type_ids_=None,
            start_positions=None, end_positions=None,
            neg_input_ids=None, neg_attention_mask=None, neg_token_type_ids=None,
            example_id=None,):
        # Context-side
        if input_ids is not None:
            assert len(input_ids.size()) == 2
            start, end = self.embed_phrase(input_ids, attention_mask, token_type_ids)

            if neg_input_ids is not None:
                neg_start, neg_end = self.embed_phrase(neg_input_ids, neg_attention_mask, neg_token_type_ids)

            # Get filter logits
            filter_output = start[:]
            filter_start_logits, filter_end_logits = self.filter_linear(filter_output).chunk(2, dim=2)
            filter_start_logits = filter_start_logits.squeeze(2)
            filter_end_logits = filter_end_logits.squeeze(2)

            if self.return_phrase:
                return (start, end, filter_start_logits, filter_end_logits)

        # Query-side
        if input_ids_ is not None:
            assert len(input_ids_.size()) == 2
            query_start, query_end = self.embed_query(input_ids_, attention_mask_, token_type_ids_)

            if self.return_query:
                return (query_start, query_end)
        
        # Get dense logits
        start_logits = start.matmul(query_start.transpose(1, 2)).squeeze(-1)
        end_logits = end.matmul(query_end.transpose(1, 2)).squeeze(-1)
        dense_logits = start_logits.unsqueeze(2) + end_logits.unsqueeze(1)

        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)

            ignored_index = start_logits.size(1)
            start_positions.clamp_(0, ignored_index)
            end_positions.clamp_(0, ignored_index)

            # 1) Single-passage loss
            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(dense_logits.mean(2), start_positions)
            end_loss = loss_fct(dense_logits.mean(1), end_positions)
            single_loss = (start_loss + end_loss) / 2
            total_loss = single_loss

            return (total_loss,)


In [6]:
model=Encoder()

In [7]:
# Preprocessing the datasets.
# Preprocessing is slighlty different for training and evaluation.
column_names = dataset["train"].column_names

question_column_name = "question" if "question" in column_names else column_names[0]
context_column_name = "context" if "context" in column_names else column_names[1]
answer_column_name = "answers" if "answers" in column_names else column_names[2]

# Padding side determines if we do (question|context) or (context|question).
pad_on_right = tokenizer.padding_side == "right"

max_seq_length = tokenizer.model_max_length
max_query_length = 50#50
pad_on_right,max_seq_length

(True, 512)

In [8]:
# Training preprocessing
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    
    tokenized_examples = tokenizer(
        examples[context_column_name],
        truncation="only_first",
        max_length=max_seq_length,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    tokenized_questions = tokenizer(
        examples[question_column_name],
        truncation="only_first",
        max_length=max_query_length,
        return_overflowing_tokens=False,
        return_offsets_mapping=False,
        padding="max_length" 
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Inflate questions based on sample_mapping
    tokenized_examples['input_ids_'] = [tokenized_questions['input_ids'][i] for i in sample_mapping]
    tokenized_examples['token_type_ids_'] = [tokenized_questions['token_type_ids'][i] for i in sample_mapping]
    tokenized_examples['attention_mask_'] = [tokenized_questions['attention_mask'][i] for i in sample_mapping]

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        #context_index = 1 if pad_on_right and args.append_title else 0
        context_index =  0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != context_index:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != context_index:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    #bugfix: limit to 1000. Ajeitar!!!
    tokenized_examples["input_ids"]=tokenized_examples["input_ids"][:1000]
    tokenized_examples["input_ids_"]=tokenized_examples["input_ids_"][:1000]
    tokenized_examples["attention_mask"]=tokenized_examples["attention_mask"][:1000]
    tokenized_examples["attention_mask_"]=tokenized_examples["attention_mask_"][:1000]
    tokenized_examples["token_type_ids"]=tokenized_examples["token_type_ids"][:1000]
    tokenized_examples["token_type_ids_"]=tokenized_examples["token_type_ids_"][:1000]
    tokenized_examples["start_positions"]=tokenized_examples["start_positions"][:1000]
    tokenized_examples["end_positions"]=tokenized_examples["end_positions"][:1000]
    return tokenized_examples

In [9]:
train_dataset = dataset["train"]
train_dataset = train_dataset.map(
                prepare_train_features,
                batched=True,
                num_proc=1,
                #remove_columns=column_names,
                #load_from_cache_file=not args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

Running tokenizer on train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Running tokenizer on train dataset: 100%|██████████| 4000/4000 [00:00<00:00, 4517.06 examples/s]


In [10]:
from transformers import default_data_collator
from transformers import DataCollatorWithPadding,EvalPrediction
from datasets import load_metric
from utils.utils_qa import postprocess_qa_predictions

# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval", filter_threshold=-1e5):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions, save_rate = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=args.version_2_with_negative,
        n_best_size=args.n_best_size,
        max_answer_length=args.max_answer_length,
        null_score_diff_threshold=args.null_score_diff_threshold,
        output_dir=args.output_dir,
        log_level=0,
        prefix=stage,
        filter_threshold=filter_threshold,
    )
    # Format the result to the format the metric expects.
    if args.version_2_with_negative:
        formatted_predictions = [
            {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        ]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

    references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references), save_rate

In [11]:
from utils.trainer_qa import QuestionAnsweringTrainer
os.environ["WANDB_DISABLED"] = "true"

pad_to_max_length=True
metric = load_metric("squad")
# Data collator
# We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data
# collator.
data_collator = (
    default_data_collator
    # if args.pad_to_max_length
    # else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
    if pad_to_max_length
    else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
)

def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

# Initialize our Trainer
trainer = QuestionAnsweringTrainer(
    model=model,
    args=args,
    # args=None,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_2503/3832162653.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_t

In [12]:
train_result = trainer.train(resume_from_checkpoint=False)

 20%|██        | 500/2500 [01:02<04:11,  7.95it/s]Checkpoint destination directory bert-finetuned-squad/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 4.7711, 'learning_rate': 1.6064e-05, 'epoch': 1.0}


 40%|████      | 1000/2500 [02:08<02:58,  8.43it/s]

{'loss': 1.0071, 'learning_rate': 1.2064e-05, 'epoch': 2.0}


 60%|██████    | 1500/2500 [03:13<02:08,  7.80it/s]

...

{'loss': 0.2555, 'learning_rate': 4.064e-06, 'epoch': 4.0}


100%|██████████| 2500/2500 [05:25<00:00,  7.93it/s]

{'loss': 0.1397, 'learning_rate': 6.4e-08, 'epoch': 5.0}


100%|██████████| 2500/2500 [05:28<00:00,  7.61it/s]

{'train_runtime': 328.3635, 'train_samples_per_second': 60.908, 'train_steps_per_second': 7.614, 'train_loss': 1.3315405059814454, 'epoch': 5.0}


### Inference

#### Create phrase vectors


In [13]:
from utils.utils_dp import convert_squad_to_hf
import logging

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

data_files = {}
data_files["test"] = convert_squad_to_hf('/home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000')

Saving 18829 (context, question, answer) triples.
Writing to /home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000_hf.json



In [14]:
extension = data_files["test"].split(".")[-1]
raw_datasets = load_dataset(extension, data_files=data_files, field="data")

column_names = raw_datasets["test"].column_names
context_column_name = "context" if "context" in column_names else column_names[1]

# Padding side determines if we do (question|context) or (context|question).
pad_on_right = tokenizer.padding_side == "right"
max_seq_length = 512

Generating test split: 18829 examples [00:00, 264724.78 examples/s]


In [15]:
offset = 0

# Validation preprocessing
def prepare_validation_features(examples, indexes):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[context_column_name],
        truncation="only_first",
        max_length=max_seq_length,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" ,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    
    # Inflate doc_idxs based on sample_mapping
    tokenized_examples['doc_idx'] = [offset + examples['doc_idx'][i] for i in sample_mapping]

    # This example_id indicates the index of an original paragraph (not question id)
    tokenized_examples['example_id'] = [indexes[i] for i in sample_mapping]

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [16]:
examples = raw_datasets["test"]
accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps)

# Predict Feature Creation
with accelerator.main_process_first():
    dataset = examples.map(
        prepare_validation_features,
        batched=True,
        num_proc=1,
        with_indices=True,
        remove_columns=column_names,
        # load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on prediction dataset",
    )

# Data collator
# We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data
# collator.
data_collator = (
    default_data_collator
    )

Running tokenizer on prediction dataset:   0%|          | 0/18829 [00:00<?, ? examples/s]

Running tokenizer on prediction dataset: 100%|██████████| 18829/18829 [00:03<00:00, 5876.06 examples/s]


In [17]:
per_device_train_batch_size=8

predict_dataset_for_model = dataset.remove_columns([ "offset_mapping","doc_idx","overflow_to_sample_mapping"])
predict_dataloader = DataLoader(
    predict_dataset_for_model, shuffle=True, collate_fn=data_collator, batch_size=per_device_train_batch_size,
)

model, predict_dataloader = accelerator.prepare(model, predict_dataloader)

In [29]:
len(predict_dataloader)

2362

In [79]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

c=0
feature_id=0
all_predictions=[]
model.return_phrase=True
for step, inputs in enumerate(predict_dataloader):
    outputs=model(**inputs)

    for item_idx, (start, end, filter_start, filter_end) in enumerate(zip(*outputs)):
        a= {'example_id': inputs['example_id'][item_idx].item(),
            'feature_id': 0, # dataloader should not be shuffled
            'start': to_numpy(start),
            'end': to_numpy(end),
            'filter_start': to_numpy(filter_start),
            'filter_end': to_numpy(filter_end),
            'inputs': inputs['input_ids'][item_idx].tolist(),
        }
        all_predictions.append(a)
    feature_id+=1
    c+=1
    if c>100:
        break
outputs[0].shape,outputs[1].shape,outputs[2].shape,outputs[3].shape

(torch.Size([8, 512, 768]),
 torch.Size([8, 512, 768]),
 torch.Size([8, 512]),
 torch.Size([8, 512]))

In [34]:
len(all_predictions),all_predictions[0]['start'].shape

(808, (512, 768))

#### Search Engine

In [19]:
inputs['input_ids'].shape

torch.Size([8, 512])

In [20]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] today, mail is delivered once a day on - site to most private homes and businesses. the usps still distinguishes between city delivery ( where carriers generally walk and deliver to mailboxes hung on exterior walls or porches, or to commercial reception areas ) and rural delivery ( where carriers generally drive ). with " curbside delivery ", mailboxes are at the ends of driveways, on the nearest convenient road. " central point delivery " is used in some locations, where several nearby residences share a " cluster " of individual mailboxes in a single housing. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [21]:
df=[]
query = 'tell me about the historical spending on defense'

for i in range(1):
    df.append({'doc_idx':0, 'par_idx':0, 'title':'', 'context':query})
p='/home/miu/Projects/NLPLab/DensePhrasesMinimal/query.json'
with open(p, 'w') as f:
        json.dump({'data': df}, f)
r_d = load_dataset(extension, data_files=p, field="data")
r_d=r_d['train']
# query_inputs = prepare_features(r_d['train'])
with accelerator.main_process_first():
    query_dataset = r_d.map(
        prepare_validation_features,
        batched=True,
        num_proc=1,
        with_indices=True,
        desc="Running tokenizer on prediction dataset",
    )
query_dataset = query_dataset.remove_columns(["par_idx", "offset_mapping","doc_idx","overflow_to_sample_mapping"])
query_dataloader = DataLoader(
    query_dataset, shuffle=True, collate_fn=data_collator, batch_size=1,
)
query_dataloader = accelerator.prepare(query_dataloader)

Generating train split: 1 examples [00:00, 925.89 examples/s]
Running tokenizer on prediction dataset: 100%|██████████| 1/1 [00:00<00:00, 342.25 examples/s]


In [22]:
model.return_query=True
model.return_phrase=False

for step, query_inputs in enumerate(query_dataloader):
    #print(query_inputs)
    query_outputs=model(input_ids_=query_inputs['input_ids'],attention_mask_=query_inputs['attention_mask'])
    
    break

In [56]:
import numpy as np
def search_mips(phrases_embs,query_emb):
    print(phrases_embs.shape,query_emb.shape)
    scores=[]
    for i in range(phrases_embs.shape[0]):
        batch=[]
        for j in range(phrases_embs.shape[1]):
            # if j%10==0:
                #batch.append(phrases_embs[i][j])
            s=np.dot(phrases_embs[i][j],query_emb)
            scores.append((i,j,s))

        #reduced_embs.append(batch)
    scores=np.asarray(scores)
    return scores

In [55]:
q_start=query_outputs[0][0].cpu().detach().numpy()[0]
q_end=query_outputs[1][0].cpu().detach().numpy()[0]
print(q_start.shape,q_end.shape)

(768,) (768,)


In [80]:
# scores=search_mips(outputs[0].cpu().detach().numpy(),q_start)
search_embs=np.asarray([all_predictions[i]['start'] for i in range(len(all_predictions))])
# search_ids = np.asarray([all_predictions[i]['inputs'] for i in range(len(all_predictions))])
print(search_embs.shape)
scores=search_mips(search_embs,q_start)

(808, 512, 768)
(808, 512, 768) (768,)


In [81]:
best_score = np.argmax(scores[:,2])
i=int(scores[best_score][0])
j=int(scores[best_score][1])
best_score,scores[best_score]

(114194, array([223.        ,  18.        ,  61.62268829]))

In [59]:
# tokenizer.decode(inputs['input_ids'][i][j:])

In [85]:
inp=all_predictions[i]['inputs']
#decode
tokenizer.decode(inp[j:])

'louisiana purchase legislative timeline - - 1803 - 1804. france turned over new orleans, the historic colonial capital, on december 20, 1803, at the cabildo, with a flag - raising ceremony in the plaza de armas, now jackson square. just three weeks earlier, on november 30, 1803, spanish officials had formally conveyed the colonial lands and their administration to france. on march 9 and 10, 1804, another ceremony, commemorated as three flags day, was conducted in st. louis, to transfer ownership of upper louisiana from spain to the french first republic, and then from france to the united states. from march 10 to september 30, 1804, upper louisiana was supervised as a military district, under commandant amos stoddard. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [